In [75]:
from neo4j import GraphDatabase, basic_auth
import nxneo4j

driver = GraphDatabase.driver("bolt://localhost:7687",auth=basic_auth("neo4j", "Kaffe1234"))

In [76]:
def get_count_of_type(label, session):
    q = "MATCH (n:%s) RETURN count(n)" % label
    res = session.run(q)
    print("Added", res.value()[0], "nodes of type", label)
    
def get_count_of_relationship(label, session):
    q = "MATCH ()-[r:PHENOTYPE]-() RETURN count(*)"
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", label)

In [77]:
# Deleting all data in database
with driver.session() as session:
        session.run('''
            MATCH (n)
            DETACH DELETE n
        ''')
        print("wipeddatabase")

wipeddatabase


In [78]:
# Adding CellTypes to graph with Cell label
with driver.session() as session:
    session.run("""\
        LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/marenpg/basal_ganglia/master/csv/cell_types.csv?token=ABSKINXDM4RF6FYKDKSV5G26FFXII" AS row
        CREATE (:Cell {id: row.ID, name: row.Cell_type_name, ont:row.Ontological_identifier})
        """
    )
    get_count_of_type("Cell", session)

Added 107 nodes of type Cell


In [79]:
# Adding CellTypes to graph
cell_types = [("Neuron", "SAO:1417703748"), 
              ("Oligodendrocyte","SAO:844118448	"), 
              ("Microglia","SAO:789292116"), 
              ("Astrocyte","SAO:1394521419")
             ]
with driver.session() as session:
    for cell_name, cell_ont in cell_types:
        q = "CREATE (:CellType {{ name: '{name}', ont: '{ont}' }})".format(name=cell_name, ont=cell_ont)
        session.run(q )
        
    get_count_of_type("CellType", session)

Added 4 nodes of type CellType


In [80]:
# Adding cell phenotypes
with driver.session() as session:
    session.run("""\
        LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/marenpg/basal_ganglia/master/csv/cell_phenotypes.csv?token=ABSKINVCY7MHJG533DSSE7K6FGVDY" AS row
        CREATE (:CellPhenotype {id: row.ID, name: row.Phenotype, ont:row.Ontological_identifier})
    """)
    get_count_of_type("CellPhenotype", session)

Added 107 nodes of type CellPhenotype


In [81]:
# Adding cell phenotype categories
with driver.session() as session:
    session.run("""\
        LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/marenpg/basal_ganglia/master/csv/cell_phenotype_categories.csv?token=ABSKINRPNOGJ77IDXHLOJVC6FGVJE" AS row
        CREATE (:CellPhenotypeCategory {id:row.ID, name: row.Phenotype_category})
    """)
    get_count_of_type("CellPhenotypeCategory", session)

Added 7 nodes of type CellPhenotypeCategory


In [83]:
# Relationship between CellPhenotype and CellPhenotypeCategory
with driver.session() as session:
    session.run("""\
        LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/marenpg/basal_ganglia/master/csv/cell_phenotypes.csv?token=ABSKINVCY7MHJG533DSSE7K6FGVDY" AS row
        MATCH (a:CellPhenotype {id: row.ID})
        MATCH (c:CellPhenotypeCategory {id: row.Phenotype_category })
        MERGE (a)-[:PHENOTYPE]->(c)
    """)
    get_count_of_relationship("PHENOTYPE", session)

Added 210 relationships of type PHENOTYPE


In [ ]:
# Relationship between Cell and CellPhenotype
with driver.session() as session:
    session.run("""\
        LOAD CSV WITH HEADERS FROM "" AS row
        MATCH (a:CellPhenotype {id: row.ID})
        MATCH (c:CellPhenotypeCategory {id: row.Phenotype_category })
        MERGE (a)-[:PHENOTYPE]->(c)
    """)
    get_count_of_relationship("PHENOTYPE", session)